# Exploring some numpy tricks for use in pointcloud analysis

This notebook takes a look at how we might use [dask](https://dask.org) in our pointcloud analyses. 


In [2]:
filename = 'uhuru_s_b3_total_gcps_group1_densified_point_cloud.xyz'

In [3]:
import numpy as np
import pandas as pd
# import xarray as xr
import dask.dataframe as dd
import dask.array as da
import multiprocessing
# import dask.array as da
# import pptk
from PDAL import PDAL

In [4]:
# from dask.distributed import Client
# client = Client()  # start local workers as processes

In [6]:
%%time
blocksize = 100e6
df = dd.read_csv(filename, blocksize=blocksize)
pcloud_np = df.to_dask_array(lengths=True)
xy = pcloud_np.T[:2]
xy = xy.T

CPU times: user 658 ms, sys: 116 ms, total: 774 ms
Wall time: 6.79 s


## Find the ConvexHull that surrounds these points

## Now iterate through the ConvexHull points to find the minimum bounding rectangle

In [8]:
import numpy as np
from scipy.spatial import ConvexHull

def minimum_bounding_rectangle(points):
    """
    Find the smallest bounding rectangle for a set of points.
    Returns a set of points representing the corners of the bounding box.

    :param points: an nx2 matrix of coordinates
    :rval: an nx2 matrix of coordinates
    """
    from scipy.ndimage.interpolation import rotate
    pi2 = np.pi/2.

    # get the convex hull for the points
    hull_points = points[ConvexHull(points).vertices]

    # calculate edge angles
    edges = np.zeros((len(hull_points)-1, 2))
    edges = hull_points[1:] - hull_points[:-1]

    angles = np.zeros((len(edges)))
    angles = np.arctan2(edges[:, 1], edges[:, 0])

    angles = np.abs(np.mod(angles, pi2))
    angles = np.unique(angles)

    # find rotation matrices
    rotations = np.vstack([
        np.cos(angles),
        np.cos(angles-pi2),
        np.cos(angles+pi2),
        np.cos(angles)]).T

    rotations = rotations.reshape((-1, 2, 2))

    # apply rotations to the hull
    rot_points = np.dot(rotations, hull_points.T)

    # find the bounding points
    min_x = np.nanmin(rot_points[:, 0], axis=1)
    max_x = np.nanmax(rot_points[:, 0], axis=1)
    min_y = np.nanmin(rot_points[:, 1], axis=1)
    max_y = np.nanmax(rot_points[:, 1], axis=1)

    # find the box with the best area
    areas = (max_x - min_x) * (max_y - min_y)
    best_idx = np.argmin(areas)

    # return the best box
    x1 = max_x[best_idx]
    x2 = min_x[best_idx]
    y1 = max_y[best_idx]
    y2 = min_y[best_idx]
    r = rotations[best_idx]

    rval = np.zeros((4, 2))
    rval[0] = np.dot([x1, y2], r)
    rval[1] = np.dot([x2, y2], r)
    rval[2] = np.dot([x2, y1], r)
    rval[3] = np.dot([x1, y1], r)
    
    return rval

In [9]:
%%time
corners = minimum_bounding_rectangle(xy)

CPU times: user 6.37 s, sys: 1.55 s, total: 7.92 s
Wall time: 20.7 s


### Using these four corners, we can now determine the rotation and translation matricies

In [10]:
from PDAL import PDAL

r_mat = np.array(PDAL.rotation_matrix(corners, dim=2))
t_mat = np.array(PDAL.translation_matrix(corners, dim=2))
angle = PDAL.rotation_angle(corners, unit='degrees')
ll = np.array(PDAL.lower_left(corners))

### Let's create a lamdba function to generate x' and y'

In [11]:
def t(x,y):
    return da.matmul(t_mat,da.matmul(r_mat,[x, y, 1]))[0:2]

In [ ]:
%%time
tdata = df.map_partitions(lambda df: df.apply((lambda row: t(row.X, row.Y)), axis=1))\
.compute(scheduler='processes')

### Pre-processing Steps

1. Perform an affine transformation on the data so that the lower left corner is in the origin, and the data is orthogonal
1. Build an r-tree for spatial mapping
1. Use tree to thin points based on nearest neighbor distance 

### Generate a new, transposed array

This array will contain only a list of all the `X` values and a list of all the `Y` values.

Uses the [np.T](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.T.html) command

### Discritize the array into the desired resolution

In [ ]:
resolution = .2 # Target resolution in meters.
xy = pcloud_np.T[:2]
xy = ((xy + resolution / 2) // resolution).astype(int)

### Find the min and max values

In [ ]:
mn, mx = xy.min(axis=1), xy.max(axis=1)
sz = mx + 1 - mn

In [ ]:
# Map the xy locations into a single index for faster access
flatidx = np.ravel_multi_index(xy-mn[:, None], sz.compute())
# Sort the index values, returning sorted index locations, not values
z_order = np.argsort(flatidx)

z_reordered = pcloud_np[z_order,2]
sorted_idx = flatidx[z_order]
bin_boundaries = np.where(sorted_idx[:-1] != sorted_idx[1:])[0]

In [ ]:
max_height = np.maximum.reduceat(z_reordered.compute(), bin_boundaries)
min_height = np.minimum.reduceat(z_reordered.compute(), bin_boundaries)
print("Min Heights: average:{avg:5.2f}, max:{maximum:5.2f}, min:{minimum:5.2f}".format(
    avg=min_height.mean(),
    maximum=min_height.max(),
    minimum=min_height.min())
     )
print("Max Heights: average:{avg:5.2f}, max:{maximum:5.2f}, min:{minimum:5.2f}".format(
    avg=max_height.mean(),
    maximum=max_height.max(),
    minimum=max_height.min())
     )

In [ ]:
v = pptk.viewer(pcloud_np)
v.set(point_size=0.001)